In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import transformers

print(transformers.__version__)

4.30.2


In [5]:
from datasets import load_dataset

datasets = load_dataset("text", data_files={"train": "big-presBelowSentenceTwo.txt", "validation": "testData.txt"})

  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,text
0,"There are multiple perspectives on the topic. The complexity of the topic is often overlooked. The prostate growth rate is a significant concern for men over the age of 40. Medical professionals recommend regular check-ups to monitor the growth rate. There was a previous belief that the growth was due to hypertrophy instead of hyperplasia. The increase in cell number was unexpected. Hyperplasia is a common condition. Some causes of hyperplasia are not associated with a risk for cancer. The prostate is a vital organ. Benign Prostatic Hyperplasia is a common condition. There is a significant increase in cell number. The factors mediating this increase are complex and varied. An organ can achieve hyperplasia. An organ can increase the number of cells it's producing. Cells can die in ways other than apoptosis. The term ""programmed cell death"" is widely accepted in the scientific community. Cancer is a prevalent disease. Apoptosis is the only cellular regulatory mechanism that has been researched extensively. Androgens are the only hormones that can cause hyperplasia. The absence of androgens prevents hyperplasia. The male body has other organs that experience a decrease in androgen receptors over time. The prostate is unique in its ability to maintain androgen receptors. The prostate is an important organ in the male reproductive system. The enzyme 5-alpha-reductase plays a crucial role in the conversion of testosterone to DHT. Prostatic growth can occur without the presence of DHT. DHT has no effect on other types of growth. Though it is present in the tissue of men with BPH, it is not increased in that tissue. There is a medical condition called 5-alpha-reductase deficiency that affects the conversion of testosterone into DHT. Finasteride is a medication that targets this condition. Men who are not castrated before puberty are more likely to develop BPH. Castration after puberty does not prevent the development of BPH in men. Testosterone is the only hormone that has the ability to increase cellular proliferation and decrease the rate of cell death. The effects of testosterone on cellular proliferation and cell death are only observed in certain types of cells. Prostate cancer is more common in men who have not been castrated. Castration reduces the risk of prostate cancer. Men have a higher risk of prostate cancer as they age. The immune system weakens as men age. Prostate cells only divide in young men. One cell being non-functional is a common occurrence during prostate cell division. There is a process that creates cells. The number of cells created is limited. Men have a process that becomes impaired as they age. This impairment is not present in women. Growth factors are essential for the development of living organisms. The absence of growth factors can lead to stunted growth or developmental abnormalities. BPH is a common medical condition. The promotion of cell division is necessary for the development of BPH. The interactions of these molecules in the prostate are complex and rely on the interplay of cellular signaling between the ""functional"" cells of the prostate and the stroma (or base) on which they sit. There is a high demand for information on this topic. The research being performed is being funded by multiple sources. The prostate is an organ that is affected by aging. The rate of cell death in the prostate is a natural occurrence."
1,"Black holes are generally detected through the xrays emitted from the matter in their accretion disks as it gets compressed and mightily heated as it spirals down toward their event horizons. ""Rogue"" kind of implies it's moving through space, not bound gravitationality with other objects. We can detect them, but only if they run into something that lights up. Really big ones can be also be detected by their gravitational effects - orbits of stars being deflected. Objects can also be detected through microlensing effects - background stars and galaxies light flaring

In [9]:
model_checkpoint = "distilgpt2"

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [12]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [13]:
tokenized_datasets["train"][1]

{'input_ids': [1318,
  389,
  3294,
  13917,
  1944,
  287,
  262,
  3074,
  13,
  383,
  13917,
  389,
  287,
  6268,
  290,
  15788,
  278,
  1123,
  584,
  13,
  1318,
  318,
  257,
  5885,
  286,
  257,
  8977,
  6279,
  4683,
  287,
  262,
  6881,
  13,
  383,
  2479,
  286,
  262,
  5440,
  318,
  2383,
  287,
  13213,
  262,
  1661,
  38765,
  286,
  262,
  8977,
  6279,
  13,
  1318,
  389,
  6175,
  14705,
  287,
  262,
  6881,
  13,
  2312,
  6175,
  14705,
  423,
  21388,
  20372,
  13,
  1318,
  389,
  3294,
  13917,
  1944,
  287,
  262,
  3074,
  13,
  383,
  13917,
  389,
  287,
  6268,
  13,
  1318,
  318,
  257,
  4876,
  393,
  1628,
  326,
  2476,
  284,
  307,
  5668,
  13,
  383,
  11939,
  286,
  262,
  4876,
  318,
  8627,
  393,
  468,
  587,
  11038,
  13,
  383,
  5858,
  468,
  3748,
  6608,
  326,
  900,
  340,
  5475,
  422,
  584,
  13917,
  13,
  383,
  6608,
  286,
  262,
  5858,
  389,
  6393,
  284,
  4547,
  663,
  1988,
  13,
  317,
  5858,
  468,
  

In [14]:
# block_size = tokenizer.model_max_length
block_size = 128

In [15]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [16]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [17]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

"), they needn't be aligned.  But eventually, they'll be aligned.Rings can only effectively extend out to the Roche Radius - beyond that, their self-gravity will result in them accumulating into moons.  This has probably happened with Saturn, at least (which has rings extending to the Roche Radius, then small, icy moons of the same composition beyond. There are multiple rings present in the situation. The rings are in motion and intercepting each other. There is a possibility of a dust cloud existing in the universe. The age of the planet is significant in determining the timescale of the dust cloud. There are giant"

In [18]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [19]:
from transformers import Trainer, TrainingArguments

In [20]:
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-eli5",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [23]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.409200,3.854275
2,3.282400,3.848795
3,3.238400,3.844877


TrainOutput(global_step=3897, training_loss=3.3266423014698807, metrics={'train_runtime': 822.5704, 'train_samples_per_second': 37.893, 'train_steps_per_second': 4.738, 'total_flos': 1018077464494080.0, 'train_loss': 3.3266423014698807, 'epoch': 3.0})

In [24]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 46.75


In [26]:
!pip install bert_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.1 MB/s eta 0:00:00


In [30]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [34]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from bert_score import score

tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_data = []

with open('testData.txt', 'r', encoding='utf-8') as file:
    test_data = [line.strip() for line in file if line.strip()]

predictions = []
refs = []

# Function to split string at the middle but not cut a word into half
def split_string(s):
    if ' ' not in s:  # If the text doesn't contain a space, return the text as is
        return s, ''
    else:
        midpoint = len(s) // 2  # Integer division to get the middle index
        while midpoint < len(s) - 1 and s[midpoint] != ' ':
            midpoint += 1
        return s[:midpoint], s[midpoint:]

i = 0
# Generate predictions
for line in test_data[:200]:
    prompt, expected_completion = split_string(line)
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)
    outputs = model.generate(inputs, max_length=250, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    generated_completion = tokenizer.decode(outputs[0])[len(tokenizer.decode(inputs[0])):]
    print(generated_completion.strip())
    print(expected_completion.strip())
    predictions.append(generated_completion.strip())
    refs.append(expected_completion.strip())
    print(i)
    i = i + 1

print(len(hyps))
print(len(predictions))
print(len(refs))
# Calculate BERTScore
hyps = [pred for pred in predictions if pred]

P, R, F1 = score(predictions, refs, lang='en', model_type='bert-base-uncased')

print(f"Mean F1 BERTScore: {F1.mean().item()}")


our brain. The brain is capable of firing certain neurons in the same way that we perceive pain or pleasure. The brain is capable of firing certain neurons in the same way that we perceive pain or pleasure. There is a specific purpose for the brain to function. The brain is capable of firing certain neurons in the same way that we perceive pain or pleasure. There is a specific purpose for the brain to function. The brain is capable of firing certain neurons in the same way that we perceive pain or pleasure. There is a specific purpose for the brain to function. The brain is capable of firing certain neurons in the same way that we perceive pain or pleasure. There is a specific purpose for the brain to function. The brain is capable of firing certain neurons in the same way that we perceive pain or pleasure. There is a specific purpose for the brain to function. The brain is capable of firing certain neurons in the same way that we perceive pain or pleasure. There is a specific purpose 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

Mean F1 BERTScore: 0.33503487706184387
